# Notebook that enables the preprocessing of the MIMIC-III dataset
- How to execute preprocessing of clinical notes: Cell -> Run all
- The utility functions can be found in folder: /src/tools/utils_preprocess.py

In [1]:
# Built-in libraries
import os
import sys
import pickle

# warnings are switched since they are not important for preprocessing
import warnings
warnings.filterwarnings("ignore")

try:
    root = os.path.dirname(os.path.dirname(oa.path.abspath(__file__)))
except NameError:
    root = os.path.dirname(os.getcwd())
sys.path.append(root)
print('Project root: {}'.format(root))

Project root: /Users/cmetzner/Desktop/Study/PhD/research/ORNL/Biostatistics and Multiscale System Modeling/attention_mechanisms


In [2]:
# Installed libraries
import pandas as pd

# custom libraries
from src.tools.utils_preprocess import rel2abs, preproc_clinical_notes
from src.tools.utils_preprocess import create_splits, get_class_type
#from src.tools.label_description import get_code_desc

[nltk_data] Downloading package punkt to /Users/cmetzner/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Define some constants

Subsets:
- 'full': Full set of available codes in MIMIC-III
- '50': 50 most frequent codes in MIMIC-III

In [3]:
SEED = 42
TRAIN_SIZE = 0.71
TEST_SIZE = 0.52
MIN_FREQ = 3

subsets = ['MimicFull', 'Mimic50']
splits = ['train', 'test', 'val']
caml = True  # Use splits and preprocessing published by Mullenbach et al. 2018 (https://github.com/jamesmullenbach/caml-mimic)

# Change paths to location of raw data (PROCEDURES_ICD.csv and DIAGNOSES_ICD.csv)
path_data_raw = os.path.join(root, 'data', 'raw')
# Change paths to location of processed data
path_data_proc = os.path.join(root, 'data', 'processed')
# Change paths to location where code descriptions are stored (CMS32_DESC_LONG_SHORT_DX.xlsx and CMS32_DESC_LONG_SHORT_SG.xlsx)
path_data_external = os.path.join(root, 'data', 'external')


# Create directories to store dataset specific data
for subset in subsets:
    if not os.path.exists(os.path.join(path_data_proc, f'data_{subset}/')):
        os.makedirs(os.path.join(path_data_proc, f'data_{subset}/'))

# Preprocess ICD-9 Procedure/Diagnoses Codes

In [4]:
# Load ICD-9 procedure and diagnosis codes
df_proc = pd.read_csv(os.path.join(path_data_raw, 'PROCEDURES_ICD.csv'))
df_diag = pd.read_csv(os.path.join(path_data_raw, 'DIAGNOSES_ICD.csv'))

# Remove all rows that have no ICD9-codes
df_proc = df_proc[~df_proc['ICD9_CODE'].isna()]
df_diag = df_diag[~df_diag['ICD9_CODE'].isna()]

print(f'Shape of procedure data: {df_proc.shape}')
print(f"Number of unique patient ids (SUBJECT_ID): {df_proc.SUBJECT_ID.nunique()}")
print(f"Number of unique hospital admission ids (HADM_ID): {df_proc.HADM_ID.nunique()}")
print(f"Number of unique ICD-9 procedure codes: {df_proc.ICD9_CODE.nunique()}")
print(f'Shape of diagnosis data: {df_diag.shape}')
print(f"Number of unique patient ids (SUBJECT_ID): {df_diag.SUBJECT_ID.nunique()}")
print(f"Number of unique hospital admission ids (HADM_ID): {df_diag.HADM_ID.nunique()}")
print(f"Number of unique ICD-9 diagnosis codes: {df_diag.ICD9_CODE.nunique()}")

print(f'\nTotal number of ICD-9 codes in raw data of MIMIC-III: {df_proc.ICD9_CODE.nunique() + df_diag.ICD9_CODE.nunique()}')

Shape of procedure data: (240095, 5)
Number of unique patient ids (SUBJECT_ID): 42214
Number of unique hospital admission ids (HADM_ID): 52243
Number of unique ICD-9 procedure codes: 2009
Shape of diagnosis data: (651000, 5)
Number of unique patient ids (SUBJECT_ID): 46517
Number of unique hospital admission ids (HADM_ID): 58929
Number of unique ICD-9 diagnosis codes: 6984

Total number of ICD-9 codes in raw data of MIMIC-III: 8993


In [5]:
# Make sure ICD-9 codes are strings
df_proc['ICD9_CODE'] = df_proc['ICD9_CODE'].astype(str)
df_diag['ICD9_CODE'] = df_diag['ICD9_CODE'].astype(str)

# Remove all whitespace
df_proc['ICD9_CODE'] = df_proc.apply(lambda x: x.ICD9_CODE.strip(), axis=1)
df_diag['ICD9_CODE'] = df_diag.apply(lambda x: x.ICD9_CODE.strip(), axis=1)

# Transform relative ICD-9 code representation to absolute code
df_proc['ABS_CODE'] = df_proc.apply(lambda x: rel2abs(x.ICD9_CODE, flag_proc=True), axis=1)
df_diag['ABS_CODE'] = df_diag.apply(lambda x: rel2abs(x.ICD9_CODE, flag_proc=False), axis=1)

df_proc['ICD9_TYPE'] = 'procedure'
df_diag['ICD9_TYPE'] = 'diagnosis'

In [6]:
# Concat dataframes containing procedure and diagnosis codes
df_codes = pd.concat([df_diag, df_proc])

In [7]:
# Save codes to csv file
df_codes.to_csv(os.path.join(path_data_proc, 'ALL_CODES.csv'), index=False,
                columns=['SUBJECT_ID', 'HADM_ID', 'ABS_CODE', 'ICD9_TYPE'],
                header=['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE', 'ICD9_TYPE'])

## Check number of labels

In [8]:
codes = pd.read_csv(os.path.join(path_data_proc, 'ALL_CODES.csv'), dtype={'ICD9_CODE': str})
codes.head(3)

,SUBJECT_ID,HADM_ID,ICD9_CODE,ICD9_TYPE
0,109,172335,403.01,diagnosis
1,109,172335,486,diagnosis
2,109,172335,582.81,diagnosis


In [9]:
# Billable ICD-9 codes
codes.ICD9_CODE.nunique()

8993

# Preprocess and tokenize clinical notes

In [10]:
# Load clinical notes dataset
notes = pd.read_csv(os.path.join(path_data_raw, 'NOTEEVENTS.csv'))
notes = notes.loc[notes.CATEGORY == 'Discharge summary']
notes = notes[['SUBJECT_ID', 'HADM_ID', 'TEXT']]

In [11]:
# Preprocess clinical notes
notes = preproc_clinical_notes(df_notes=notes, path_data_proc=path_data_proc, caml_clean=caml)

  0%|          | 0/52726 [00:00<?, ?it/s]

## Take a Look at the cleaned and tokenized clinical notes

In [12]:
notes = pd.read_pickle(os.path.join(path_data_proc, 'CLEANED_NOTES.pkl'))

In [13]:
notes.head(5)

,SUBJECT_ID,HADM_ID,TEXT
0,22532,167853.0,"[admission, date, :, deidentified, discharge, ..."
1,13702,107527.0,"[admission, date, :, deidentified, discharge, ..."
2,13702,167118.0,"[admission, date, :, deidentified, discharge, ..."
3,13702,196489.0,"[admission, date, :, deidentified, discharge, ..."
4,26880,135453.0,"[admission, date, :, deidentified, discharge, ..."


# Combine Labels with Clinical Notes
- Investigate all procedure codes that have just 2 positions and see if we can change them to a more meaningful version


To-Do's
- Split datasets into full, 50
- Split SUBJECT_ID and HADM_ID into train, test, val - split is based on SUBJECT_ID --> Avoid data leakage
- Retrieve categories (everything before the .)
- Create X (map token2idx)
- Retrieve list of labels BASED ON given DATASET
- Create y (map codes and cats to multi-hot vectors)

In [14]:
# Number of hospital admissions with clinical notes
print(f'Number of hospital admission ids with clinical notes: {notes.shape[0]}')

Number of hospital admission ids with clinical notes: 52726


In [15]:
# 1. Align HADM_IDS of codes and notes
# Some HADM_ID have no codes but no notes
# Some HADM_ID have notes but no codes (we already filtered out the HADM_ID without codes)

# Get all hospital admission ids that have notes
hadm_notes = notes.HADM_ID.unique().tolist()

# Retrieve codes for those hospital admission ids
codes = codes[codes['HADM_ID'].isin(hadm_notes)]

# Check number of hospital admission id's in codes dataset
print(f'Number of hospital admission ids that have codes and notes: {codes.HADM_ID.nunique()}')

# Apparently, we have 4 hospital admission id's that have notes but no codes
# Let's filter them out.
# Get hospital admission ids with notes but are not included in the codes dataframe
display(notes[~notes['HADM_ID'].isin(codes.HADM_ID.unique().tolist())])

hadm_notes_wo_codes = notes[~notes['HADM_ID'].isin(codes.HADM_ID.unique().tolist())].HADM_ID.tolist()
print(hadm_notes_wo_codes)

Number of hospital admission ids that have codes and notes: 52722


,SUBJECT_ID,HADM_ID,TEXT
16238,13567,110220.0,"[admission, date, :, deidentified, discharge, ..."
21393,31866,182252.0,"[admission, date, :, deidentified, discharge, ..."
23115,24975,109963.0,"[admission, date, :, deidentified, discharge, ..."
32306,17796,142890.0,"[admission, date, :, deidentified, discharge, ..."


[110220.0, 182252.0, 109963.0, 142890.0]


In [16]:
notes = notes[~notes['HADM_ID'].isin(hadm_notes_wo_codes)]

In [17]:
# 1. Check if both dataframes have same number of hospital admission id's
print('Check if number of HADM_IDs are equal in both dataframes.')
print(f'Codes == Notes : {codes.HADM_ID.nunique()} == {notes.HADM_ID.nunique()}')

Check if number of HADM_IDs are equal in both dataframes.
Codes == Notes : 52722 == 52722


# Create datasets for experiments by subsetting HADM_ID
- full: 
- 50

## Dataset: 'full' set of codes in MIMIC-III:

In [18]:
codes_full = codes.copy()

In [19]:
l_codes_full = sorted(codes_full.ICD9_CODE.unique().tolist())

In [20]:
print(f'Number of billable code labels: {len(l_codes_full)}')

Number of billable code labels: 8907


In [21]:
with open(os.path.join(path_data_proc, f'data_MimicFull', 'l_codes_MimicFull.pkl'), 'wb') as f:
    pickle.dump(l_codes_full, f)

In [22]:
# Aggregate individual label mentions in lists
codes_full = codes_full.groupby(['SUBJECT_ID', 'HADM_ID']).agg({'ICD9_CODE': list}).reset_index()

# Remove redundant labels in lists
codes_full['ICD9_CODE'] = codes_full.apply(lambda x: sorted(list(set(x.ICD9_CODE))), axis=1)

In [23]:
# Generate full dataset
df_full = codes_full.merge(notes, how='left', on=['SUBJECT_ID', 'HADM_ID']).sort_values(['SUBJECT_ID', 'HADM_ID'])

# Save dataset
df_full.to_pickle(os.path.join(path_data_proc, 'data_MimicFull', 'DATA_MimicFull.pkl'))

## Dataset: n='50' most frequent in MIMIC-III:

In [24]:
n = 50
# Retrieve the 50 most frequent codes in MIMIC-III
most_50 = sorted(codes.ICD9_CODE.value_counts()[:n].index.tolist())
most_50

['038.9',
 '244.9',
 '250.00',
 '272.0',
 '272.4',
 '276.1',
 '276.2',
 '285.1',
 '285.9',
 '287.5',
 '305.1',
 '311',
 '33.24',
 '36.15',
 '37.22',
 '38.91',
 '38.93',
 '39.61',
 '39.95',
 '401.9',
 '403.90',
 '410.71',
 '412',
 '414.01',
 '424.0',
 '427.31',
 '428.0',
 '45.13',
 '486',
 '496',
 '507.0',
 '511.9',
 '518.81',
 '530.81',
 '584.9',
 '585.9',
 '599.0',
 '88.56',
 '88.72',
 '93.90',
 '96.04',
 '96.6',
 '96.71',
 '96.72',
 '99.04',
 '99.15',
 '995.92',
 'V15.82',
 'V45.81',
 'V58.61']

In [25]:
# Subset codes dataset
codes_50 = codes.copy()
codes_50 = codes_50[codes_50['ICD9_CODE'].isin(most_50)]

In [26]:
# Get list with labels
l_codes_50 = sorted(codes_50.ICD9_CODE.unique().tolist())
print(f'Number of billable code labels: {len(l_codes_50)}')

# Save
with open(os.path.join(path_data_proc, 'data_Mimic50', 'l_codes_Mimic50.pkl'), 'wb') as f:
    pickle.dump(l_codes_50, f)

Number of billable code labels: 50


In [27]:
# Aggregate individual label mentions in lists
codes_50 = codes_50.groupby(['SUBJECT_ID', 'HADM_ID']).agg({'ICD9_CODE': list}).reset_index()

# Remove redundant labels in lists
codes_50['ICD9_CODE'] = codes_50.apply(lambda x: sorted(list(set(x.ICD9_CODE))), axis=1)

In [28]:
# Generate full dataset
df_50 = codes_50.merge(notes, how='left', on=['SUBJECT_ID', 'HADM_ID']).sort_values(['SUBJECT_ID', 'HADM_ID'])

# Save dataset
df_50.to_pickle(os.path.join(path_data_proc, 'data_Mimic50', 'DATA_Mimic50.pkl'))

# Generate training/testing/validation splits (X and y samples)
In this segment we are splitting the four individual datasets into training/testing/validation sets using the patient ids (SUBJECT_ID) and HADM_ID. By using SUBJECT_ID we are avoiding data leackage from one split to another, i.e., one distinct patient only occurs in either one of those splits.

For sake of reproducibility, for the `full` and `50` datasets we are using the published HADM_ID's proposed by Mullenbach et al. 2018, since a plethora of published work used them as the foundation (source: https://github.com/jamesmullenbach/caml-mimic/tree/master/mimicdata/mimic3).

In [29]:
for subset in subsets:
    create_splits(subset=subset, path_data_proc=path_data_proc, min_freq=MIN_FREQ)


Current subset: MimicFull
Current split: train
Vocab size for train: 120481
Current split: test
Current split: val

Current subset: Mimic50
Current split: train
Vocab size for train: 50176
Current split: test
Current split: val


# Get Code Descriptions

In [ ]:
#for subset in subsets:            
#    get_code_desc(path_data_external=path_data_external, path_data_processed=path_data_proc, subset=subset)